In [1]:
%load_ext autoreload
%autoreload 2

In [16]:
from gptchem.extractor import FewShotClassificationExtractor
from gptchem.formatter import FewShotFormatter
from gptchem.data import get_photoswitch_data

import pandas as pd 
import numpy as np

from sklearn.model_selection import train_test_split

from langchain.llms import Anthropic

In [12]:
num_classes = 2
train_size = 20
seed = 42
max_test_size = 50
representation = 'SMILES'

In [8]:
data = get_photoswitch_data()
data = data.dropna(subset=["E isomer pi-pi* wavelength in nm"])
data["binned"] = pd.qcut(
    data["E isomer pi-pi* wavelength in nm"], num_classes, labels=np.arange(num_classes)
)
train, test = train_test_split(
    data, train_size=train_size, random_state=seed, stratify=data["binned"]
)

In [9]:
data = get_photoswitch_data()
data = data.dropna(subset=["E isomer pi-pi* wavelength in nm"])
data["binned"] = pd.qcut(
    data["E isomer pi-pi* wavelength in nm"], num_classes, labels=np.arange(num_classes)
)
train, test = train_test_split(
    data, train_size=train_size, random_state=seed, stratify=data["binned"]
)

In [13]:
formatter = FewShotFormatter(
    train,
    property_name="transition wavelength",
    representation_column=representation,
    label_column="binned",
)

In [54]:
formatter._PREFIX = "You are a highly intelligent chemisty question answering bot that answers questions about {property}. Answer by simply returning only 0 or 1, no other text is necessary."

In [55]:
test = test.sample(min(len(test), max_test_size), random_state=seed)
formatted = formatter(test)

In [56]:
llm = Anthropic(anthropic_api_key='sk-PxG6E4m8pVJbFXGutrSAe1WB68sNYz3uI8fB9wzfiA-Z1OeqUyFm5F5yx3k34i4xyNyrmY8WgXnoChH-Rx-_mQ', temperature=0)

In [57]:
llm(formatted.iloc[0]['prompt'])

'\nQ: OC%42=C%43N=CC=CC%43=C(/N=N/C%44=NC%45=CC=CC=C%45N%44)C=C%42\nA: 0'

In [58]:
extractor = FewShotClassificationExtractor()

In [59]:
pred = '\nQ: C1(/N=N/C2=CC=NN2)=CC=NN1\nA: 0\n\nQ: Cn1c(C)c(c2c1cccc2)N=Nc1nncs1\nA: 1\n\nQ: O=[N+]([O-])C1=CC(/N=N/C2=CC=CC=C2)=CC=C1\nA: 0\n\nQ: [H]C1=CC([N+]([O-])=O)=CC(C#N)=C1/N=N/C2=CC([H])=C(C=C2[H])N(CC)CC\nA: 1\n\nQ: Cc1[nH]c2c(c1N=Nc1nncs1)cccc2\nA: 1\n\nQ: CC(C=C(N(CCC#N)CCO)C=C1)=C1/N=N/C2=CC=CC=C2\nA: 1\n\nQ: BrC1=CC=C(/N=N/C2=CC=CC=C2'

In [60]:
for i, row in formatted.iterrows():
    if i > 10:
        break
    pred = llm(row['prompt'])

    extracted = extractor.extract(pred.replace(row['representation'], ''))
    print(extracted, row['label'])

0 1
0 1
1 1
0 0
1 1
1 1
1 1
1 1
1 0
1 1
1 1
